# PROJET IA for HumanForYou - Entraînement et Évaluation du Modèle

|Auteur|
|---|
|G. DUBOYS DE LAVIGERIE|
|T. VILLETTE|
|O. BOUSSARD|
|A. BRICON|

## Objectifs du Notebook

Ce notebook a pour objectif de développer plusieurs modèles d'IA en utilisant les données prétraitées et de les évaluer sur un ensemble de test. Les étapes comprennent :

1. **Division des données :** Séparation des données en ensembles d'entraînement et de test.
2. **Entraînement des modèles :** Développement de plusieurs modèles d'IA avec les données d'entraînement.
3. **Évaluation des modèles :** Mesure des performances de chaque modèle sur l'ensemble de test.
4. **Sélection du meilleur modèle :** Analyse des résultats pour choisir le modèle offrant les performances les plus prometteuses.

## Attendus

À la fin de ce notebook, le modèle sera prêt à être déployé, accompagné d'une évaluation détaillée de ses performances.

## Prérequis

Avant d'exécuter ce notebook, assurez-vous d'avoir exécuté le notebook `data_preprocessing.ipynb` pour garantir que les données sont prêtes pour l'entraînement du modèle.

### Préparation de l'environnement

In [ ]:
# Compatibilité entre Python 2 et Python 3
from __future__ import division, print_function, unicode_literals

# Imports nécessaires
import numpy as np
import os

# Assurer la stabilité du notebook entre différentes exécutions
np.random.seed(42)

# Affichage de figures directement dans le notebook
%matplotlib inline
import matplotlib.pyplot as plt

# Ajustements des paramètres de l'affichage des figures
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Ignorer les avertissements inutiles (voir SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

### Importation des données traitées réalisée dans le notebook `data_preprocessing.ipynb`

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None) # Retirer la limite de colonnes à afficher

def load_preprocessed_data(file_path='../datasets/data_preprocessing.csv'):
    return pd.read_csv(file_path)

# Charger les données prétraitées
result = load_preprocessed_data()
result.head()

### Division des données (jeu d'entraînement, jeu de test)
Cette étape consiste à séparer les données en un jeu d'entraînement (80%) et un jeu de test (20%) afin d'évaluer la capacité du modèle à généraliser sur de nouvelles données. 

- Le **jeu d'entraînement** est utilisé pour former le modèle, lui fournissant une quantité suffisante de données pour apprendre les motifs et les relations présents dans nos données.

- Le **jeu de test** permet de mesurer la performance du modèle sur des données non vues pendant l'entraînement, offrant ainsi une évaluation impartiale de sa capacité de généralisation.

Le choix de la division 80/20 n'est pas arbitraire. Il s'agit d'une règle empirique bien établie dans le domaine de l'apprentissage machine, offrant un équilibre optimal entre la taille de l'ensemble d'entraînement nécessaire pour un apprentissage efficace et la capacité à évaluer le modèle de manière significative sur le jeu de test.


In [ ]:
from sklearn.model_selection import train_test_split

# Supposons que nous voulons prédire 'cat_nom__Attrition_Yes'
y = result['Attrition']
X = result.drop(['Attrition'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Diagramme pour le jeu d'entraînement
plt.figure(figsize=(8, 5))
y_train.value_counts().plot(kind='bar', color=['blue', 'red'])
plt.title('Répartition de Attrition dans le jeu d\'entraînement')
plt.xlabel('Attrition')
plt.ylabel('Nombre d\'employés')
plt.xticks(rotation=0)
plt.show()

# Diagramme pour le jeu de test
plt.figure(figsize=(8, 5))
y_test.value_counts().plot(kind='bar', color=['blue', 'red'])
plt.title('Répartition de Attrition dans le jeu de test')
plt.xlabel('Attrition')
plt.ylabel('Nombre d\'employés')
plt.xticks(rotation=0)
plt.show()



In [ ]:
# Affichage d'un graphique
fig, ax = plt.subplots(figsize=(2, 5))

rect_train = plt.Rectangle((0, 0), 1, 0.8, color='red', alpha=0.5, label='Entraînement')
rect_test = plt.Rectangle((0, 0.8), 1, 0.2, color='blue', alpha=0.5, label='Test')

ax.add_patch(rect_train)
ax.add_patch(rect_test)

ax.set_xlim(0, 1)
ax.set_ylim(0, 1)
ax.set_xticks([])
ax.set_yticks([])
ax.set_title("Répartition des Données")

ax.text(0.5, 0.4, f'{len(X_train)}', ha='center', va='center', color='black', fontsize=8)
ax.text(0.5, 0.9, f'{len(X_test)}', ha='center', va='center', color='black', fontsize=8)

ax.legend(loc='center')

plt.show()


### Choisir et Entraîner un modèle

Avant de plonger dans les détails, il est essentiel de comprendre comment choisir, entraîner et évaluer un modèle. Une étape cruciale consiste à analyser les performances du modèle sur un ensemble de test et à utiliser des techniques comme la validation croisée pour garantir une évaluation robuste du modèle.

**Fonction de validation croisée**

La fonction `train_evaluate_model` ci-dessous illustre un processus générique d'entraînement et d'évaluation de modèles de régression. Cela garantit une approche standardisée dans notre notebook, simplifiant ainsi le processus et assurant une cohérence d'évaluation, indépendamment de l'algorithme sous-jacent.

En utilisant la fonction `train_evaluate_model`, nous appliquons la validation croisée avec deux métriques :
- **RMSE (Root Mean Square Error)** : Mesure la moyenne des erreurs au carré entre prédictions et valeurs réelles, avec prise de la racine carrée de cette moyenne.
- **MAE (Mean Absolute Error)** : Autre métrique mesurant la moyenne des valeurs absolues des erreurs entre prédictions et valeurs réelles.

Cette approche renforce l'évaluation des modèles en utilisant `scikit-learn` pour calculer le RMSE, la MAE, et la validation croisée, fournissant une évaluation complète de la performance du modèle.

Il convient de noter que nous utilisons cette fonction pour chacun de nos modèles, assurant ainsi une comparaison équitable des performances entre les différentes approches algorithmiques que nous explorons.

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

def train_model(trainer):
    # Entraînement du modèle
    trainer.fit(X_train, y_train)
    
    some_data = X_train.iloc[:5]
    some_labels = y_train.iloc[:5]

    # Et on effectue la prédiction :
    print("Predictions:", trainer.predict(some_data))
    print("Labels:", list(some_labels)) # vraies valeurs

    # Prédictions sur les données de test
    trainer_predictions = trainer.predict(X_test)
    # Évaluation des prédictions sur les données de test
    trainer_mse = mean_squared_error(y_test, trainer_predictions)
    trainer_rmse = np.sqrt(trainer_mse)

    # Affichage des métriques d'évaluation
    print("MSE:", trainer_mse)
    print("RMSE:", trainer_rmse)

    # Validation croisée pour évaluer la performance du modèle
    scores = cross_val_score(trainer, X_test, y_test,
                             scoring="neg_mean_squared_error", cv=10)
    trainer_rmse_scores = np.sqrt(-scores)

    # Affichage des scores de validation croisée
    print("Cross-Validation Scores:", trainer_rmse_scores)

    # Affichage de la performance moyenne et de la variabilité du modèle
    print("Mean RMSE:", trainer_rmse_scores.mean())
    print("RMSE Standard Deviation:", trainer_rmse_scores.std())

#### Modèle de régression

<u>Modèle de régression linéaire</u>

Entraînons un modèle de [_régression linéaire_](https://fr.wikipedia.org/wiki/R%C3%A9gression_lin%C3%A9aire) avec nos données.

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
train_model(lin_reg)

<u>Modèle de régression par arbre de décision (DecisionTreeRegressor) </u>

In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Create a DecisionTreeRegressor
tree_reg = DecisionTreeRegressor(random_state=42)
train_model(tree_reg)


<u>Modèle RandomForestRegressor</u>

In [ ]:
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(max_depth=15, max_features='log2', n_estimators=26,
                      random_state=42)
train_model(forest_reg)

#### Modèle de classification

In [ ]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, precision_recall_curve, roc_curve
from sklearn.model_selection import cross_val_predict

# Affiche le graphique de la précision et recall en fonction du seuil
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.figure(figsize=(8, 4))
    plt.plot(thresholds, precisions[:-1], "b-", label="Precision", linewidth=2)
    plt.plot(thresholds, recalls[:-1], "g-", label="Recall", linewidth=2)
    plt.xlabel("Threshold", fontsize=16)
    plt.legend(loc="upper left", fontsize=16)
    plt.show()

# Affiche le graphique de la précision en fonction du recall
def plot_precision_vs_recall(precisions, recalls):
    plt.figure(figsize=(8, 6))
    plt.plot(recalls, precisions, "k-", linewidth=2)
    plt.xlabel("Recall", fontsize=16)
    plt.ylabel("Precision", fontsize=16)
    plt.show()

# Entraîner et évaluer le modèle puis afficher les scores
def display_scores(model, x, y, method):
    model.fit(x, y) # Entraîner le modèle

    y_train_pred = cross_val_predict(model, x, y, cv=3) # Validation croisée
    y_scores = cross_val_predict(model, x, y, cv=5, method=method) # Calcul des scores

    # Calcul des différents indicateurs de performance
    conf_matrix = confusion_matrix(y, y_train_pred)
    precision = precision_score(y, y_train_pred)
    recall = recall_score(y, y_train_pred)
    f1 = f1_score(y, y_train_pred)
    accuracy = accuracy_score(y, y_train_pred)


    # Si method est "decision_function", y_scores est un tableau 1D
    if method == "decision_function":
        # calculer précisions, recalls et seuils pour la courbe ROC
        precisions, recalls, thresholds = precision_recall_curve(y, y_scores)
        a, b, c = roc_curve(y, y_scores)
    # Si method est "predict_proba", y_scores est un tableau 2D
    else:
        # calculer précisions, recalls et seuils pour la courbe ROC
        precisions, recalls, thresholds = precision_recall_curve(y, y_scores[:, -1])
        a, b, c = roc_curve(y, y_scores[:, -1])

    # Affichage des indicateurs de performance
    print('Matrice de confusion : ', conf_matrix)
    print('Précision : ', precision)
    print('Recall : ', recall)
    print('F1-score : ', f1)
    print('Accuracy : ', accuracy)

    # Affichage des graphiques
    plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
    plot_precision_vs_recall(precisions, recalls)
    
    # retourner précisions, recalls et seuils pour la courbe ROC
    return a, b, c
    


Mélange de données

In [ ]:
import numpy as np

shuffle_index = np.random.permutation(len(X_train))
X_train, y_train = X_train.iloc[shuffle_index], y_train.iloc[shuffle_index]

# Garder les données avec attrition à True (> 0)
y_train_yes = (y_train > 0)
y_test_yes = (y_test > 0 )

##### SDG Classifier

In [ ]:
from sklearn.linear_model import SGDClassifier

# Créer un SGDClassifier
sgd_clf = SGDClassifier(max_iter=1000, random_state=42)

# Entraîner et évaluer le modèle
fpr_sdg, tpr_sdg, thresholds_sdg = display_scores(sgd_clf, X_train, y_train_yes, "decision_function")

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Créer un RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42, n_estimators=10)

# Entraîner et évaluer le modèle
fpr_clf, tpr_clf, thresholds_clf = display_scores(forest_clf, X_train, y_train_yes, "predict_proba")

Régression logistique

In [ ]:
from sklearn.linear_model import LogisticRegression

# Créer une LogistiqueRegression
logisticRegr = LogisticRegression()

# Entraîner et évaluer le modèle
fpr_lgr, tpr_lgr, thresholds_lgr = display_scores(logisticRegr, X_train, y_train_yes, "predict_proba")

Voting

In [ ]:
from sklearn.ensemble import  VotingClassifier

# Créer un RandomForestClassifier et un LogisticRegression
voting_forest = RandomForestClassifier(random_state=42, n_estimators=10)
voting_logisticRegr = LogisticRegression()

# Créer un VotingClassifier en mixant le RandomForestClassifier et la LogisticRegression
voting_clf = VotingClassifier(estimators=[('lr', voting_logisticRegr), ('rf', voting_forest)], voting='soft')

# Entraîner et évaluer le modèle
fpr_voting, tpr_voting, thresholds_voting = display_scores(voting_clf, X_train, y_train_yes, "predict_proba")

#### Comparer les modèles 
Courbe ROC

In [ ]:
from sklearn.metrics import roc_curve

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate', fontsize=16)
    plt.ylabel('True Positive Rate', fontsize=16)

# Afficher le graphique
plt.figure(figsize=(8, 6))
plt.plot(fpr_lgr, tpr_lgr, "b:", linewidth=2, label="Régression logistique")
plot_roc_curve(fpr_voting, tpr_voting, "Voting")
plot_roc_curve(fpr_clf, tpr_clf, "Random Forest")
plot_roc_curve(fpr_sdg, tpr_sdg, "SDG Classifier")
plt.legend(loc="lower right", fontsize=16)
plt.show()

## Gridsearch
Utilisation de la méthode GridSearchCV afin de trouver les meilleurs hyperparamètres pour un modèle de régression RandomForestRegressor.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# Définir les paramètres
param_grid = {
    'n_estimators': [50],
    'max_depth': [None, 10],
    'min_samples_split': [2],
    'min_samples_leaf': [1],
    'bootstrap': [True, False]
}

# Créer un GridSearchCV
grid_search = GridSearchCV(forest_clf, param_grid, cv=StratifiedKFold(n_splits=3), scoring='accuracy', verbose=1, n_jobs=-1)

# Effectuer la recherche par grille sur les données d'entraînement
grid_search.fit(X_train, y_train_yes)

# Afficher les meilleurs paramètres et la performance associée
print("Meilleurs paramètres : ", grid_search.best_params_)
print("Meilleur score de validation croisée : ", grid_search.best_score_)


In [ ]:
# Récupérer l'importance de chaque paramètre pour notre modèle
feature_importances = grid_search.best_estimator_.feature_importances_
# Afficher l'importance de chaque paramètre
sorted(zip(feature_importances, X.columns), reverse=True)

In [ ]:
indices = np.argsort(feature_importances)

# Afficher le graphique
plt.style.use('fivethirtyeight')
%matplotlib inline
plt.figure(figsize=(25,25))

plt.title('Feature Importances')
plt.barh(range(len(indices)), feature_importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [X.columns[i] for i in indices])
plt.xlabel('Relative Importance')

In [ ]:

y_test_pred = grid_search.best_estimator_.predict(X_test)

conf_matrix = confusion_matrix(y_test_yes, y_test_pred)
precision = precision_score(y_test_yes, y_test_pred)
recall = recall_score(y_test_yes, y_test_pred)
f1 = f1_score(y_test_yes, y_test_pred)
accuracy = accuracy_score(y_test_yes, y_test_pred)

# Affichage des indicateurs de performance
print('Matrice de confusion : ', conf_matrix)
print('Précision : ', precision)
print('Recall : ', recall)
print('F1-score : ', f1)
print('Accuracy : ', accuracy)
